# Omics Cookbook

## Library Imports

Run this cell to import the necessary libraries

In [2]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re

import cptac.endometrial as en
import cptac.algorithms as al

Welcome to the cptac data service package. Available datasets may be
viewed using cptac.list_data(). In order to access a specific data
set, import a cptac subfolder using either 'import cptac.dataset' or
'from cptac import dataset'.
******
Version: 0.4.0
******
You have loaded the cptac endometrial dataset. To view available
dataframes, use cptac.endometrial.list_data(). To view available
functions for accessing and manipulating the dataframes, use
cptac.endometrial.list_api().
endometrial data version: 2.1

Loading Dictionary...
Loading cptac endometrial data:
Loading proteomics data...
Loading clinical data...
Loading acetylproteomics data...
Loading phosphoproteomics_site data...
Loading somatic data...
Loading transcriptomics_circular data...
Loading phosphoproteomics_gene data...
Loading transcriptomics_linear data...
Loading miRNA data...
Loading somatic_binary data...
Loading CNA data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available

## Get frequently mutated genes list

Enter the type of cancer and the cutoff for mutation frequency that you would like to use.

In [2]:
cancer_type = "Endometrial"
desired_cutoff = .15

fm = al.get_frequently_mutated(cancer_type, cutoff=desired_cutoff)
print(fm)

       Gene  Percent Mutated
0     RPL22         0.168421
1    ARID1A         0.452632
2    PIK3CA         0.494737
3     FBXW7         0.189474
4     RBM27         0.168421
5      NSD1         0.168421
6     SYNE1         0.168421
7     ZFHX4         0.200000
8      PTEN         0.789474
9      CTCF         0.284211
10   ZNF469         0.157895
11     TP53         0.221053
12    KMT2B         0.242105
13     KRAS         0.326316
14    LMAN1         0.157895
15    SCAF4         0.168421
16   PIK3R1         0.389474
17   CTNNB1         0.305263
18     PCLO         0.168421
19    KMT2D         0.178947
20    HERC2         0.178947
21    ZFHX3         0.221053
22     JAK1         0.168421
23    OBSCN         0.157895
24    AHNAK         0.157895
25  CCDC168         0.168421
26    DOCK3         0.200000
27    MUC16         0.189474
28    HUWE1         0.157895
29   DNAH17         0.157895
30   INPPL1         0.200000


## Select a gene from the list of frequently mutated genes above

Set the gene to which of the above frequently mutated genes you want to examine. For example, if you want to look at the gene PTEN, change the cell below to say "gene = 'PTEN'"

In [44]:
gene = 'ARID1A'

## Select desired omics comparison

Select which type of omics you want to compare. If you want to compare proteomics data for the interacting proteins, uncomment the "omics = en.get_proteomics()" line, etc.

In [64]:
#omics = en.get_proteomics()
#omics = en.get_transcriptomics()
omics = en.get_phosphoproteomics()
#omics = en.get_acetylproteomics()

## Generate interacting proteins and test omics comparisons

Simply run this cell after following the instructions above to see the results of the comparisons

In [39]:
try:
    '''Prevent slice degrees of freedom warning'''
    import warnings
    warnings.filterwarnings("ignore")

    print("\nGene: ", gene)

    '''Use get interacting proteins method to generate list of interacting proteins'''
    interacting_proteins = al.get_interacting_proteins(gene)

    print("Generating interacting protein list")
    interacting_proteins_in_omics_df = []

    '''Only do comparisons on proteins in the omics dataframe'''
    for ip in interacting_proteins:
        if omics.name == 'phosphoproteomics' or omics.name == 'acetylproteomics':
            col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
        else:
            col_regex = '^{}$'.format(ip)

        selected = omics.filter(regex=col_regex)

        if len(selected.columns) > 0:
            interacting_proteins_in_omics_df.append(ip)

    '''Ceate dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.append_mutations_to_omics(mutation_genes=[gene], omics_df=omics, omics_genes=interacting_proteins_in_omics_df)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")

    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except:
    print("Error in Comparison")


Gene:  ZFHX3
Generating interacting protein list
Doing t-test comparisons

         Comparison   P_Value
0  ZFHX3_proteomics  0.001988





## Look at effect on all proteins

This will look at the effect of the selected gene mutation on all proteins, and report any significant results.

In [61]:
try:
    '''Prevent slice degrees of freedom warning'''
    #import warnings
    #warnings.filterwarnings("ignore")

    print("\nGene: ", gene)

    '''Use get interacting proteins method to generate list of interacting proteins'''
    proteomics = en.get_proteomics()
    proteins = proteomics.columns
    proteins = proteins[:300]

    print("Generating protein list")
    interacting_proteins_in_omics_df = []

    '''Only do comparisons on proteins in the omics dataframe'''
    for ip in proteins:
        if omics.name == 'phosphoproteomics' or omics.name == 'acetylproteomics':
            col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
        else:
            col_regex = '^{}$'.format(ip)

        selected = omics.filter(regex=col_regex)

        if len(selected.columns) > 0:
            interacting_proteins_in_omics_df.append(ip)

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.append_mutations_to_omics(mutation_genes=[gene], omics_df=omics, omics_genes=interacting_proteins_in_omics_df)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    #print(protdf)

    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  ARID1A
Generating protein list
Error in Comparison
columns overlap but no suffix specified: Index(['ABHD14A-ACY1-K374_acetylproteomics'], dtype='object')


In [55]:
print(protdf)

           A1BG_transcriptomics  A2M_transcriptomics  A2ML1_transcriptomics  \
Sample_ID                                                                     
S001                       4.02                13.39                   6.79   
S002                       4.81                13.24                   6.33   
S003                       6.24                14.32                   9.42   
S005                       5.31                13.77                   4.22   
S006                       9.84                13.12                   6.83   
S007                       5.03                14.13                   7.75   
S008                       3.17                13.49                   5.88   
S009                       6.03                13.67                   6.69   
S010                       6.02                14.30                   8.47   
S011                       5.71                14.97                   9.96   
S012                       5.17                13.40

In [62]:
proteomics = en.get_proteomics()
proteins = proteomics.columns
proteins = proteins[0:300]

print(proteins)
print(len(proteins))

interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in proteins:
    if omics.name == 'phosphoproteomics' or omics.name == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print(interacting_proteins_in_omics_df)
print(len(interacting_proteins_in_omics_df))

protdf = en.append_mutations_to_omics(mutation_genes=[gene], omics_df=omics, omics_genes=interacting_proteins_in_omics_df)
protdf.head()

Index(['A1BG', 'A2M', 'A2ML1', 'A4GALT', 'AAAS', 'AACS', 'AADAT', 'AAED1',
       'AAGAB', 'AAK1',
       ...
       'AKAP1', 'AKAP10', 'AKAP11', 'AKAP12', 'AKAP13', 'AKAP17A', 'AKAP5',
       'AKAP7', 'AKAP8', 'AKAP8L'],
      dtype='object', name='idx', length=300)
300
['A2M', 'AACS', 'AAGAB', 'AAK1', 'AARS', 'AASS', 'AATF', 'ABAT', 'ABCB7', 'ABCE1', 'ABCF1', 'ABHD10', 'ABHD11', 'ABHD14A', 'ABHD14A-ACY1', 'ABHD16A', 'ABI3BP', 'ABLIM1', 'ABLIM3', 'ABRACL', 'ABRAXAS2', 'ACAA1', 'ACAA2', 'ACAD11', 'ACAD8', 'ACAD9', 'ACADM', 'ACADS', 'ACADSB', 'ACADVL', 'ACAP2', 'ACAT1', 'ACAT2', 'ACIN1', 'ACLY', 'ACO1', 'ACO2', 'ACOT11', 'ACOT13', 'ACOT4', 'ACOT7', 'ACOT8', 'ACOT9', 'ACOX1', 'ACOX3', 'ACP1', 'ACSF2', 'ACSL3', 'ACSL4', 'ACSL5', 'ACSM3', 'ACSS1', 'ACSS2', 'ACSS3', 'ACTA1', 'ACTB', 'ACTL6A', 'ACTN1', 'ACTN2', 'ACTN4', 'ACTR1A', 'ACTR1B', 'ACTR2', 'ACTR3', 'ACTR6', 'ACTR8', 'ACYP1', 'ACYP2', 'ADAM10', 'ADAR', 'ADD1', 'ADD3', 'ADH1B', 'ADH1C', 'ADH5', 'ADH7', 'ADI1', 'ADIRF', 'ADK', 'ADM', '

ValueError: columns overlap but no suffix specified: Index(['ABHD14A-ACY1-K374_acetylproteomics'], dtype='object')

In [8]:
import cptac.endometrial as en

proteomics = en.get_proteomics()
proteins = proteomics.columns
proteins = proteins[0:220]
gene = "ARID1A"
phos = en.get_phosphoproteomics()

protdf = en.append_mutations_to_omics(mutation_genes=[gene], omics_df=phos, omics_genes=proteins)
print(protdf)

A1BG did not match any columns in phosphoproteomics dataframe. A1BG_phosphoproteomics column inserted, but filled with NaN.
A2M did not match any columns in phosphoproteomics dataframe. A2M_phosphoproteomics column inserted, but filled with NaN.
A2ML1 did not match any columns in phosphoproteomics dataframe. A2ML1_phosphoproteomics column inserted, but filled with NaN.
A4GALT did not match any columns in phosphoproteomics dataframe. A4GALT_phosphoproteomics column inserted, but filled with NaN.
AADAT did not match any columns in phosphoproteomics dataframe. AADAT_phosphoproteomics column inserted, but filled with NaN.
AAMP did not match any columns in phosphoproteomics dataframe. AAMP_phosphoproteomics column inserted, but filled with NaN.
AAR2 did not match any columns in phosphoproteomics dataframe. AAR2_phosphoproteomics column inserted, but filled with NaN.
AARS2 did not match any columns in phosphoproteomics dataframe. AARS2_phosphoproteomics column inserted, but filled with NaN.


ACSL1 did not match any columns in phosphoproteomics dataframe. ACSL1_phosphoproteomics column inserted, but filled with NaN.
ACSM3 did not match any columns in phosphoproteomics dataframe. ACSM3_phosphoproteomics column inserted, but filled with NaN.
ACSS1 did not match any columns in phosphoproteomics dataframe. ACSS1_phosphoproteomics column inserted, but filled with NaN.
ACTA1 did not match any columns in phosphoproteomics dataframe. ACTA1_phosphoproteomics column inserted, but filled with NaN.
ACTA2 did not match any columns in phosphoproteomics dataframe. ACTA2_phosphoproteomics column inserted, but filled with NaN.
ACTB did not match any columns in phosphoproteomics dataframe. ACTB_phosphoproteomics column inserted, but filled with NaN.
ACTG1 did not match any columns in phosphoproteomics dataframe. ACTG1_phosphoproteomics column inserted, but filled with NaN.
ACTG2 did not match any columns in phosphoproteomics dataframe. ACTG2_phosphoproteomics column inserted, but filled with

ValueError: columns overlap but no suffix specified: Index(['ACADM-S240_phosphoproteomics'], dtype='object')